<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # Import pandas for data handling and plotting

# --- 1. Environmental & Global Parameters ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=50): # Assuming 50 steps per day
        self.total_sim_steps = total_sim_steps
        self.days_per_cycle = days_per_cycle
        self.time = 0 # Current time step (index)

        # --- UV Profile with Seasonal Variation ---
        # Base daily UV cycle (sine wave, from 0 to 1)
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle))
        daily_uv_cycle[daily_uv_cycle < 0] = 0 # Only positive UV, night is 0

        # Seasonal modulation (yearly sine wave)
        # Assuming a 365-day year, total_sim_steps needs to cover this.
        # Amplitude of seasonal variation: from 0.5 (winter min) to 1.0 (summer max)
        # This makes summer peaks 1.0 and winter peaks 0.5 * 1.0 = 0.5
        seasonal_amplitude_mod = 0.25 * np.sin(np.linspace(0, 2 * np.pi * (total_sim_steps / (365 * days_per_cycle)), total_sim_steps)) + 0.75
        # This makes seasonal_amplitude_mod range from 0.5 (0.75-0.25) to 1.0 (0.75+0.25)

        # Combine daily cycle with seasonal modulation
        # Create the full UV intensity array for the entire simulation
        self.uv_intensity = np.zeros(total_sim_steps)
        for i in range(total_sim_steps):
            day_step_in_cycle = i % days_per_cycle
            self.uv_intensity[i] = daily_uv_cycle[day_step_in_cycle] * seasonal_amplitude_mod[i]

    def get_current_uv(self):
        if self.time >= self.total_sim_steps:
            return 0.0
        return self.uv_intensity[self.time]

    def step(self):
        self.time += 1
        self.current_day_step = self.time % self.days_per_cycle

# --- 2. Proto-Cell/System Class ---
class ProtoCell:
    def __init__(self, env):
        self.env = env

        # Core Molecular Pools (as concentrations/amounts)
        self.rna_pool = 100.0 # General RNA (templates, proto-rRNA)
        self.dna_pool = 10.0  # Basic DNA (more stable template)
        self.atp_pool = 50.0  # Energy currency
        self.mcyw_peptide_pool = 5.0 # Foraging/Antenna peptides
        self.other_peptide_pool = 2.0 # Non-mcyw peptides

        # Resource Stocks (internal to the proto-cell)
        self.internal_ntp_stock = 100.0
        self.internal_aa_stock = 100.0
        self.internal_trp_tyr_cys_stock = 3.0 # DECREASED for more scarcity (was 5.0)

        # System States & Damage
        self.rna_damage_level = 0.0 # Accumulates from UV
        self.protein_synthesis_active = True # Whether ribosomes are active
        self.rna_dna_copying_active = True # Whether replication is active
        self.uv_safety_status = "SAFE" # Based on internal signals
        self.premature_stop_signal = 0.0 # Initialize here, updated in apply_uv_damage

        # --- Parameters (tuned for MANAGED CYCLICAL ARREST AND SUSTAINABLE VIABILITY) ---
        # UV Damage parameters
        self.uv_damage_rate_per_uv = 0.18 # DECREASE SLIGHTLY: (was 0.25) Still damaging, but not overwhelming
        self.damage_tolerance_threshold = 3.5 # SLIGHTLY INCREASE: (was 3.0) Give cell a little more tolerance

        # Foraging parameters (Keep robust, ensures resources are found)
        self.foraging_efficiency_mcyw = 0.15 # Keep as is
        self.foraging_atp_cost = 0.005 # Keep as is
        self.trp_tyr_cys_foraging_bias = 8.0 # Keep as is

        # Protein Synthesis parameters (ensure mcyw production is strong for long term)
        self.protein_synthesis_rate = 0.15 # SLIGHTLY INCREASE: (was 0.12) Make proteins slightly faster
        self.protein_synthesis_ntp_cost = 0.1
        self.protein_synthesis_aa_cost = 0.1
        self.protein_synthesis_atp_cost = 0.02
        self.mcyw_synthesis_bias = 0.5 # Keep high to ensure mcyw production

        # Stop Codon Precariousness/Signaling (Keep sensitive to ensure arrest triggers)
        self.precarious_codon_rate_per_damage = 0.3 # Keep as is
        self.premature_stop_signal_threshold = 0.35 # Keep as is

        # ATP Synthesis (CRITICAL for energy surplus)
        self.atp_generation_rate_per_mcyw_uv = 0.07 # SIGNIFICANTLY INCREASE: (was 0.05) More energy from light!

        # Repair parameters (CRITICAL for coping with damage)
        self.repair_rate = 0.2 # SIGNIFICANTLY INCREASE: Make repair much faster (was 0.1)
        self.repair_atp_cost = 0.01
        self.trp_tyr_cys_for_repair_needed = 0.08 # SLIGHTLY DECREASE: (was 0.1) Make repair slightly less costly in rare AAs

        # RNA/DNA Copying parameters (Allow for overall growth now that energy/repair are better)
        self.rna_dna_copy_rate = 0.04 # INCREASE: (was 0.03) For noticeable growth over longer periods
        self.rna_dna_copy_ntp_cost = 0.05
        self.rna_dna_copy_atp_cost = 0.01

        # Degradation rates (Keep relatively low to allow for growth)
        self.rna_degradation_rate = 0.0008
        self.peptide_degradation_rate = 0.0015
        self.atp_degradation_rate = 0.003

    # --- Core Processes ---
    def _apply_uv_damage(self):
        current_uv = self.env.get_current_uv()
        if current_uv > 0:
            damage_increase = current_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            # Simulate the "safety reportage" signal from damaged precarious codons
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else:
            self.premature_stop_signal = 0 # No new stops at night

    def _forage(self):
        if self.mcyw_peptide_pool <= 0 or self.atp_pool <= self.foraging_atp_cost:
            return # Cannot forage without mcyw or ATP

        # Consume ATP for foraging movement
        atp_consumed = self.foraging_atp_cost * self.mcyw_peptide_pool
        self.atp_pool -= atp_consumed
        self.atp_pool = max(0, self.atp_pool) # Ensure not negative

        # Acquire general AAs and NTP
